### Step 1: Setting Up libraries

#### Installing 
- earthengine-api (Google Earth Engine API) for accessing Google Earth Engine
- geemap (Google Earth Engine Python API) for visualizing and analyzing Earth Engine datasets interactively in a Jupyter notebook
- geetools (Community tools for Google Earth Engine Python API) for helping with tasks such as batch processing and exporting images from Earth Engine

In [1]:
# %pip install earthengine-api geetools geemap pycrs GDAL

Upgrading Earth Engine Python API

In [2]:
# %pip install earthengine-api geetools --upgrade

In [3]:
import ee
import geemap
import geetools as gt
import numpy as np
import pandas as pd
from ipywidgets import Layout

Map = geemap.Map(layout=Layout(height='600px'))
Map.add_basemap('HYBRID')

### Step 2: Initialization

Importing libraries

In [4]:
# ee.Authenticate()

In [5]:
ee.Initialize()

Setup Variables

In [6]:
# export resolution in m/px
SCALE = 10

# name of the label 
# 0 is agri
# 1 is water
# 2 is urban
# 3 is barren
AGRI = 0
WATER = 1
URBAN = 2
BARREN = 3

# Bands to be used
BANDS = ['B2', 'B3', 'B4', 'B8']

# POINTS Location
LABELS = [
    {
        "name": "Agri",
        # path to the geojson file containing the points
        "src": "./Data/featureCollections/Agri.geojson",
        "class": AGRI
    },
    {
        "name": "Water",
        "src": "./Data/featureCollections/Water.geojson",
        "class": WATER
    },
    {
        "name": "Urban",
        "src": "./Data/featureCollections/Urban.geojson",
        "class": URBAN
    },
    {
        "name": "Barren",
        "src": "./Data/featureCollections/Barren.geojson",
        "class": BARREN
    }
]

# Visualization parameters
SFCC = {'bands': ['B8', 'B4', 'B3'], 'max': 3000}

# Drive Folder to save the data
FOLDER = "GEE"

Downloading S2 data

In [7]:
roi = geemap.shp_to_ee("./Data/sangamner_shp/sangamner.shp")
s2 = ee.ImageCollection('COPERNICUS/S2').filterDate('2022-12-15', '2023-01-20').filter(
    ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 5)).filterBounds(roi)
display(s2)
Map.centerObject(roi, 10)

Name,Description
B1,Aerosols
B2,Blue
B3,Green
B4,Red
B5,Red Edge 1
B6,Red Edge 2
B7,Red Edge 3
B8,NIR
B8A,Red Edge 4
B9,Water vapor


### Step 3: Processing Points

In [8]:
s2_list = s2.toList(s2.size())
s2_list

In [9]:
img = ee.Image(s2_list.get(12)).clip(roi)
img = img.select(BANDS)
Map.addLayer(img, SFCC, 'FCC')

In [10]:
img

In [11]:
# Importing Collections
collections = []
for label in LABELS:
    collections.append(geemap.geojson_to_ee(label['src']))

In [12]:
collections[WATER]

In [13]:
h_w = 100 #hxw meters, actually 10m extra on each dim final patch 110mx110m
# since buffer will extend 50m on each side

# Function to get the patch and the geometry from a point feature
def getPatch(feature: ee.Feature, image: ee.Image) -> tuple[ee.Image, ee.Geometry]:
    buffer_out = feature.buffer(int(h_w/2))
    buffer_out = buffer_out.bounds()
    # rectangle_region = ee.Geometry.Rectangle(buffer_out)
    patch= image.clip(buffer_out)
    return patch, buffer_out

# Function to build the bounding boxes around the point features
def buildBoundingBoxes(featureCollection: ee.FeatureCollection) -> ee.FeatureCollection:
    f = lambda feature, previous: ee.FeatureCollection(previous).merge(feature.buffer(int(h_w/2)).bounds().geometry())
    return ee.FeatureCollection(featureCollection.iterate(f, ee.FeatureCollection([])))

# Function to reformat the feature collection to add the label
def reformatFeatureCollection(featureCollection: ee.FeatureCollection, label:int) -> ee.FeatureCollection:
    newFList = []
    fSize = featureCollection.size().getInfo()
    fList = featureCollection.toList(fSize)
    for i in range(fSize):
        fGeometry = ee.Feature(fList.get(i)).geometry()
        newFeature = ee.Feature(fGeometry, {'label': label})
        newFList.append(newFeature)
    return ee.FeatureCollection(newFList)

# Function to build the image collection from the feature collection
def buildImageCollection(featureCollection: ee.FeatureCollection, image: ee.Image) -> ee.ImageCollection:
    f = lambda feature, previous: ee.ImageCollection(previous).merge(getPatch(feature, image)[0])
    return ee.ImageCollection(featureCollection.iterate(f, ee.ImageCollection([])))

# Function to map the image and the geometry
def mapImageGeometry(featureCollection: ee.FeatureCollection, image:ee.Image) -> list:
    imageGeoList = []
    fColSize = featureCollection.size().getInfo()
    fColList = featureCollection.toList(fColSize)
    for i in range(fColSize):
        feature = ee.Feature(fColList.get(i))
        patch, buffer_out = getPatch(feature, image)
        item = {'image': patch, 'bounds': buffer_out}
        imageGeoList.append(item)
    return imageGeoList

In [14]:
# build bounding boxes
bounds = []
for label in LABELS:
    bounds.append(
        reformatFeatureCollection(
            featureCollection=buildBoundingBoxes(collections[label["class"]]),
            label=label["class"]
        )
    )

In [15]:
display(bounds[AGRI].size().getInfo())
display(bounds[AGRI].first().getInfo())

332

{'type': 'Feature',
 'geometry': {'geodesic': False,
  'type': 'Polygon',
  'coordinates': [[[74.24106840549916, 19.483400836050194],
    [74.24201668673902, 19.483400836050194],
    [74.24201668673902, 19.48430054973621],
    [74.24106840549916, 19.48430054973621],
    [74.24106840549916, 19.483400836050194]]]},
 'id': '0',
 'properties': {'label': 0}}

In [16]:
def exportFilesToDrive():
    export_img_task = ee.batch.Export.image.toDrive(
        image=img, 
        description="4326", 
        folder=FOLDER, 
        scale=SCALE, 
        crs="EPSG:4326",
    )
    export_img_task.start()
    for label in LABELS:
        name = "{}_{}Bounds".format(label["class"], label["name"])
        export_task = ee.batch.Export.table.toDrive(
            collection=bounds[label["class"]],
            description=name,
            fileNamePrefix=name,
            folder=FOLDER,
            fileFormat="SHP",
        )
        export_task.start()

In [17]:
for label in LABELS:
    Map.addLayer(bounds[label["class"]], {}, "{}Bounds".format(label["name"]))
Map.centerObject(bounds[AGRI].first())

In [18]:
display(Map)

Map(center=[19.483850693201667, 74.24154254611909], controls=(WidgetControl(options=['position', 'transparent_…

In [19]:
# exportFilesToDrive()

In [ ]:
agriImages = buildImageCollection(collections[AGRI], img)
waterImages = buildImageCollection(collections[WATER], img)
buildupImages = buildImageCollection(collections[URBAN], img)
barrenImages = buildImageCollection(collections[BARREN], img)

In [ ]:
Map.addLayer(agriImages, SFCC, "agriCropped")
Map.addLayer(waterImages, SFCC, "waterCropped")
Map.addLayer(buildupImages, SFCC, "buildUpCropped")
Map.addLayer(barrenImages, SFCC, "barrenCropped")

In [ ]:
display(Map)